<a href="https://colab.research.google.com/github/muchaimaryanne/Machine-Learning-API-using-FastAPI/blob/main/Machine_Learning_API_using_FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**



In this project, we aim to create an API that can be requested to interact with a ML model. This is an interesting solution when you want to keep your model architecture secret or to make your model available to users already having an API. By creating an API, and deploying it, your model can so that receive request using the internet protocol as presented by the illustration below.

Sepsis is an infection of the blood stream resulting in a cluster of symptoms such as drop in a blood pressure, increase in heart rate and fever.

The model that we will build will be able to determine the prescence or abscence of Sepsis.




# Importation
Here is the section to import all the packages/libraries that will be used through this notebook.

In [1]:
# Data handling
import pandas as pd
import numpy as np

# Vizualisation (Matplotlib, Plotly, Seaborn, etc. )
# Machine learning libraries and metrics
# Feature Processing (Scikit-learn processing, etc. )
# Other packages



# Data Loading

Here is the section to load the datasets (train, eval, test) and the additional files

In [2]:
url = 'https://drive.google.com/file/d/1D_0MXHhkZKdbU38V9hbRRCPMwVI-_0ZZ/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

In [3]:
# Data loading
sepsis_train_df = pd.read_csv(url)

Description of the data columns.

| Column Name | Attribute/Target | Description |
|----------|----------|----------|
| ID   | N/A   | Unique number to represent patient ID   |
| PRG   | Attribute1	   | Plasma glucose   |
|PL	|Attribute 2	|Blood Work Result-1 (mu U/ml)|
|PR	|Attribute 3	|Blood Pressure (mm Hg)|
|SK	|Attribute 4	|Blood Work Result-2 (mm)|
|TS	|Attribute 5	|Blood Work Result-3 (mu U/ml)|
|M11|	Attribute 6	|Body mass index (weight in kg/(height in m)^2|


# Hypothesis

In this section we stated our null and alternate hypothesis as well as the questions that we seek to answer.

**Null Hypothesis:** 

**Alternate Hypothesis:** 

**Questions**

# Exploratory Data Analysis: EDA
Here is the section to inspect the datasets in depth, present it, make hypotheses and think the cleaning, processing and features creation.

In [5]:
sepsis_train_df.shape

(599, 11)

The data has 599 rows and 11 columns

In [4]:
sepsis_train_df.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,Sepssis
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive
1,ICU200011,1,85,66,29,0,26.6,0.351,31,0,Negative
2,ICU200012,8,183,64,0,0,23.3,0.672,32,1,Positive
3,ICU200013,1,89,66,23,94,28.1,0.167,21,1,Negative
4,ICU200014,0,137,40,35,168,43.1,2.288,33,1,Positive


In [6]:
sepsis_train_df.tail()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,Sepssis
594,ICU200604,6,123,72,45,230,33.6,0.733,34,0,Negative
595,ICU200605,0,188,82,14,185,32.0,0.682,22,1,Positive
596,ICU200606,0,67,76,0,0,45.3,0.194,46,1,Negative
597,ICU200607,1,89,24,19,25,27.8,0.559,21,0,Negative
598,ICU200608,1,173,74,0,0,36.8,0.088,38,1,Positive


In [7]:
sepsis_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         599 non-null    object 
 1   PRG        599 non-null    int64  
 2   PL         599 non-null    int64  
 3   PR         599 non-null    int64  
 4   SK         599 non-null    int64  
 5   TS         599 non-null    int64  
 6   M11        599 non-null    float64
 7   BD2        599 non-null    float64
 8   Age        599 non-null    int64  
 9   Insurance  599 non-null    int64  
 10  Sepssis    599 non-null    object 
dtypes: float64(2), int64(7), object(2)
memory usage: 51.6+ KB


In [9]:
sepsis_train_df.isna().sum()

ID           0
PRG          0
PL           0
PR           0
SK           0
TS           0
M11          0
BD2          0
Age          0
Insurance    0
Sepssis      0
dtype: int64